In [7]:
import oracledb
import pandas as pd

In [8]:
df = pd.read_csv('../data/bank_reviews_with_themes.csv')

In [ ]:
connection = oracledb.connect(
    user="bank_reviews",
    password="bank_reviews_password",
    dsn = "localhost:1521/XEPDB1" 
)
cursor = connection.cursor()

In [ ]:
# Add banks 
bank_ids = {}

for bank_name in df["bank"].unique():
    # Insert only if not already inserted
    cursor.execute("SELECT bank_id FROM banks WHERE name = :1", [bank_name])
    row = cursor.fetchone()

    if row:
        bank_ids[bank_name] = row[0]
    else:
        cursor.execute("INSERT INTO banks (name) VALUES (:1)", [bank_name])
        # Fetch the ID just inserted
        cursor.execute("SELECT bank_id FROM banks WHERE name = :1", [bank_name])
        bank_ids[bank_name] = cursor.fetchone()[0]


In [13]:
# Add reviews

cursor.execute("SELECT * FROM banks")
rows = cursor.fetchall()

for row in rows:
    print(row)

(1, 'Commercial Bank of Ethiopia')
(2, 'Bank of Abyssinia')
(3, 'Dashen Bank')


In [ ]:
# Step 2: Insert reviews
inserted = 0
for _, row in df.iterrows():
    try:
        cursor.execute("""
            INSERT INTO reviews (
                review_id, review_text, rating, review_date, bank_id, source,
                vader_sentiment, compound_score, bert_sentiment, bert_score, identified_theme
            )
            VALUES (
                :1, :2, :3, TO_DATE(:4, 'YYYY-MM-DD'), :5, :6,
                :7, :8, :9, :10, :11
            )
        """, (
            int(row["review_id"]),
            row["review"],
            int(row["rating"]),
            row["date"],
            bank_ids[row["bank"]],
            row["source"],
            row["vader_sentiment"],
            float(row["compound_score"]),
            row["bert_sentiment"],
            float(row["bert_score"]),
            row["identified_theme(s)"]
        ))
        inserted += 1
    except oracledb.DatabaseError as e:
        print("Skipping review_id", row["review_id"], "due to error:", e)

# Step 3: Commit the inserts
connection.commit()
print(f"✅ {inserted} reviews inserted.")

# Step 4: Verification – count total rows
cursor.execute("SELECT COUNT(*) FROM reviews")
count = cursor.fetchone()[0]
print("📊 Total reviews in database:", count)

# Step 5: Preview first 5 rows (joined with bank names)
cursor.execute("""
    SELECT r.review_id, b.name AS bank_name, r.rating, r.vader_sentiment, r.bert_sentiment
    FROM reviews r
    JOIN banks b ON r.bank_id = b.bank_id
    FETCH FIRST 5 ROWS ONLY
""")

print("\n📌 Sample reviews:")
for row in cursor.fetchall():
    print(row)

# Step 6: Close everything
cursor.close()
connection.close()

✅ 1648 reviews inserted.
📊 Total reviews in database: 1648

📌 Sample reviews:
(79, 'Commercial Bank of Ethiopia', 1, 'negative', 'negative')
(80, 'Commercial Bank of Ethiopia', 5, 'neutral', 'positive')
(81, 'Commercial Bank of Ethiopia', 5, 'positive', 'positive')
(82, 'Commercial Bank of Ethiopia', 5, 'positive', 'positive')
(83, 'Commercial Bank of Ethiopia', 5, 'positive', 'positive')
